# Задание 1
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание 2
* Выберите регион
* Сравните среднюю цену квадратного метра в городах этого региона
* Определите количество строящихся объектов и квартир в регионах
* Определите объем строящегося жилья в квадратных метрах

# Задание 3 *
* Напишете скрипт, который сможет ежедневно собирать данные по цене, количестве и объему строящихся объектов самостоятельно и ежедневно
* Попробуйте ускорить процесс
* Добавьте возможность добавления данных в БД с указанием текущей даты.

In [141]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import sqlite3
%matplotlib inline

## Список объектов

In [26]:
offset_ = 0
limit_ = 5000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()
objects_list = objects_data.get('data').get('list')
objids = [x.get('objId') for x in objects_list]

In [27]:
# все ли id уникальные
len(objids)==len(np.unique(objids))

True

In [77]:
objids[:7]

[13399, 13398, 13383, 13397, 13391, 23682, 36321]

# Данные по определенному объекту

In [111]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/36321' #

In [112]:
res = requests.get(url)
objprop=res.json()

In [113]:
objprop.get('data')#.get('developer')

{'address': 'г Владивосток, район Ленинский, ул 10-я Рабочая, д. 80',
 'conclusion': False,
 'developer': {'buildObjCnt': 1,
  'comissObjCnt': 0,
  'devEmail': '1962166@mail.ru',
  'devEmplMainFullNm': 'Жимолдинов Михаил Михайлович',
  'devFactAddr': 'край Приморский, г Владивосток, ул Прапорщика Комарова д. 58',
  'devFullCleanNm': 'CПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК АЗИЯ-КОНТРАКТ',
  'devId': 10195,
  'devInn': '2536319917',
  'devKpp': '253601001',
  'devLegalAddr': 'Приморский край, город Владивосток, улица Прапорщика Комарова дом 58',
  'devOgrn': '1192536030169',
  'devOrgRegRegionCd': 25,
  'devPhoneNum': '+7(914)196-21-66',
  'devShortCleanNm': 'CЗ АЗИЯ-КОНТРАКТ',
  'devShortNm': 'CЗ АЗИЯ-КОНТРАКТ',
  'devSite': 'lasto4ka.ru',
  'fundGuarantyFlg': 0,
  'govFundFlg': 0,
  'lastRpdId': 578136,
  'objGuarantyEscrowFlg': 1,
  'orgForm': {'fullForm': 'Общество с ограниченной ответственностью',
   'id': 4,
   'shortForm': 'ООО'},
  'problObjCnt': 0,
  'regRegionDesc': 'Приморский край'},


# Задание 1

* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

### Сбор данных по всем объектам и сохранение в dataframe

In [314]:
i=0
for oid in objids:
    urli = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{oid}'
    resi = requests.get(urli)
    objpropi=resi.json()

    objproplist={}
    
    if len(objpropi.get('data').get('photoRenderDTO'))!=0 and objpropi.get('data').get('photoRenderDTO')[0].get('objBuildTypeShortDesc')=='Жилое':
        pass
    else:
        continue
    
    if len(objpropi.get('data').get('photoRenderDTO'))!=0:
        objproplist['objId']=objpropi.get('data').get('photoRenderDTO')[0].get('objId')
        objproplist['objLivingType']=objpropi.get('data').get('photoRenderDTO')[0].get('objBuildTypeShortDesc')
        objproplist['objStage']=objpropi.get('data').get('photoRenderDTO')[0].get('objReadyDesc')
    else:
        objproplist['objId']=None
        objproplist['objLivingType']=None
        objproplist['objStage']=None
    
    objproplist['objDateFullReady']=objpropi.get('data').get('objReady100PercDt')
    objproplist['objRegion']=objpropi.get('data').get('developer').get('regRegionDesc')
    objproplist['objAddress']=objpropi.get('data').get('address')
    objproplist['objLkLatitude']=objpropi.get('data').get('objLkLatitude')
    objproplist['objLkLongitude']=objpropi.get('data').get('objLkLongitude')
    objproplist['objNonlivFirstFloor']=objpropi.get('data').get('nonlivFirstFloor')
    objproplist['objCompensFundFlg']=objpropi.get('data').get('objCompensFundFlg')
    objproplist['objElemLivingCnt']=objpropi.get('data').get('objElemLivingCnt')
    objproplist['objElemParkingCnt']=objpropi.get('data').get('objElemParkingCnt')
    objproplist['objElevatorCargoCnt']=objpropi.get('data').get('objElevatorCargoCnt')
    objproplist['objElevatorPassengerCnt']=objpropi.get('data').get('objElevatorPassengerCnt')
    objproplist['objElevatorWheelchairCnt']=objpropi.get('data').get('objElevatorWheelchairCnt')
    objproplist['objEnergyEffShortDesc']=objpropi.get('data').get('objEnergyEffShortDesc')
    objproplist['objFlatCnt']=objpropi.get('data').get('objFlatCnt')
    objproplist['objFlatSq']=objpropi.get('data').get('objFlatSq')
    objproplist['objFloorCnt']=objpropi.get('data').get('objFloorCnt')
    objproplist['objGreenHouseFlg']=objprop.get('data').get('objGreenHouseFlg')
    objproplist['objGuarantyEscrowFlg']=objpropi.get('data').get('objGuarantyEscrowFlg')
    objproplist['objInfrstrBicycleLaneFlg']=objpropi.get('data').get('objInfrstrBicycleLaneFlg'),
    objproplist['objInfrstrCurbLoweringFlg']=objpropi.get('data').get('objInfrstrCurbLoweringFlg')
    objproplist['objInfrstrNotObjPrkngCnt']=objpropi.get('data').get('objInfrstrNotObjPrkngCnt')
    objproplist['objInfrstrObjPrkngCnt']=objpropi.get('data').get('objInfrstrObjPrkngCnt')
    objproplist['objInfrstrPlaygrndCnt']=objpropi.get('data').get('objInfrstrPlaygrndCnt')
    objproplist['objInfrstrRampFlg']=objpropi.get('data').get('objInfrstrRampFlg')
    objproplist['objInfrstrSportGroundCnt']=objpropi.get('data').get('objInfrstrSportGroundCnt')
    objproplist['objInfrstrTrashAreaCnt']=objpropi.get('data').get('objInfrstrTrashAreaCnt')
    objproplist['objLivCeilingHeight']=objpropi.get('data').get('objLivCeilingHeight')
    objproplist['objLkClassDesc']=objpropi.get('data').get('objLkClassDesc')
    objproplist['objLkFinishTypeDesc']=objpropi.get('data').get('objLkFinishTypeDesc')
    objproplist['objLkFreePlanDesc']=objpropi.get('data').get('objLkFreePlanDesc')
    objproplist['objNonlivElemCnt']=objpropi.get('data').get('objNonlivElemCnt')
    objproplist['objPriceAvg']=objpropi.get('data').get('objPriceAvg')
    objproplist['objProblemFlg']=objpropi.get('data').get('objProblemFlg')
    objproplist['objSquareLiving']=objpropi.get('data').get('objSquareLiving')
    objproplist['objStatus']=objpropi.get('data').get('objStatus')
    objproplist['wallMaterialShortDesc']=objpropi.get('data').get('wallMaterialShortDesc')

    if i==0:
        objdf=pd.DataFrame(objproplist)
    else:
        objdf=pd.concat([objdf,pd.DataFrame(objproplist)],ignore_index=True)
    i+=1

In [315]:
objdf.head()

,objId,objLivingType,objStage,objDateFullReady,objRegion,objAddress,objLkLatitude,objLkLongitude,objNonlivFirstFloor,objCompensFundFlg,...,objLivCeilingHeight,objLkClassDesc,objLkFinishTypeDesc,objLkFreePlanDesc,objNonlivElemCnt,objPriceAvg,objProblemFlg,objSquareLiving,objStatus,wallMaterialShortDesc
0,13398,Жилое,Строится,2024-12-31,Тюменская область,"г Тюмень, ул Профсоюзная, д. 56",57.1519,65.5544,0,0,...,"2,96",Элитный,Без отделки,Нет,48,47886,0,4812.98,0,Другое
1,13383,Жилое,Строится,2022-06-30,Тюменская область,г Тюмень,57.153,65.5343,1,1,...,"2,8",Элитный,По желанию заказчика,Есть,10,53109,0,6289.98,0,Другое
2,13397,Жилое,Строится,2024-12-31,Тюменская область,"г Тюмень, ул Профсоюзная, д. 56",57.1519,65.5544,0,0,...,"2,96",Элитный,Без отделки,Нет,70,45523,0,6248.51,0,Другое
3,13391,Жилое,Строится,2022-06-30,Тюменская область,"г Тюмень, ул Профсоюзная",57.1541,65.5595,1,1,...,"2,8",Элитный,По желанию заказчика,Есть,26,55893,0,4293.67,0,Другое
4,23682,Жилое,Строится,2022-06-30,Санкт-Петербург город,д Агалатово,60.2152,30.3089,0,1,...,"2,73",Типовой,Без отделки,Нет,1,54859,0,2748.15,0,Блоки


In [316]:
objdf.dtypes

objId                          int64
objLivingType                 object
objStage                      object
objDateFullReady              object
objRegion                     object
objAddress                    object
objLkLatitude                 object
objLkLongitude                object
objNonlivFirstFloor           object
objCompensFundFlg              int64
objElemLivingCnt               int64
objElemParkingCnt              int64
objElevatorCargoCnt           object
objElevatorPassengerCnt       object
objElevatorWheelchairCnt      object
objEnergyEffShortDesc         object
objFlatCnt                     int64
objFlatSq                     object
objFloorCnt                    int64
objGreenHouseFlg               int64
objGuarantyEscrowFlg           int64
objInfrstrBicycleLaneFlg       int64
objInfrstrCurbLoweringFlg      int64
objInfrstrNotObjPrkngCnt      object
objInfrstrObjPrkngCnt         object
objInfrstrPlaygrndCnt         object
objInfrstrRampFlg              int64
o

In [317]:
objdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4464 entries, 0 to 4463
Data columns (total 39 columns):
objId                        4464 non-null int64
objLivingType                4464 non-null object
objStage                     4464 non-null object
objDateFullReady             4464 non-null object
objRegion                    4428 non-null object
objAddress                   4446 non-null object
objLkLatitude                4464 non-null object
objLkLongitude               4464 non-null object
objNonlivFirstFloor          4438 non-null object
objCompensFundFlg            4464 non-null int64
objElemLivingCnt             4464 non-null int64
objElemParkingCnt            4464 non-null int64
objElevatorCargoCnt          4395 non-null object
objElevatorPassengerCnt      4395 non-null object
objElevatorWheelchairCnt     4395 non-null object
objEnergyEffShortDesc        4464 non-null object
objFlatCnt                   4464 non-null int64
objFlatSq                    4464 non-null objec

In [318]:
objdf['objId']=objdf['objId'].map(lambda x: int(x) if x is not None else None)

#objdf['objDateFullReady']=pd.to_datetime(objdf['objDateFullReady'],format='%Y-%m-%d')

objdf['objLkLatitude']=objdf['objLkLatitude'].map(lambda x: float(x) if x is not None else None)

objdf['objLkLongitude']=objdf['objLkLongitude'].map(lambda x: float(x) if x is not None else None)

objdf['objPriceAvg']=objdf['objPriceAvg'].map(lambda x: float(x) if x is not None else None)

objdf['objFlatSq']=objdf['objFlatSq'].map(lambda x: float(x) if x is not None else None)

objdf['objLivCeilingHeight']=objdf['objLivCeilingHeight'].map(lambda x: float(x.split(' ')[0].replace(',','.')) if x is not None else None)

In [319]:
objdf.dtypes

objId                          int64
objLivingType                 object
objStage                      object
objDateFullReady              object
objRegion                     object
objAddress                    object
objLkLatitude                float64
objLkLongitude               float64
objNonlivFirstFloor           object
objCompensFundFlg              int64
objElemLivingCnt               int64
objElemParkingCnt              int64
objElevatorCargoCnt           object
objElevatorPassengerCnt       object
objElevatorWheelchairCnt      object
objEnergyEffShortDesc         object
objFlatCnt                     int64
objFlatSq                    float64
objFloorCnt                    int64
objGreenHouseFlg               int64
objGuarantyEscrowFlg           int64
objInfrstrBicycleLaneFlg       int64
objInfrstrCurbLoweringFlg      int64
objInfrstrNotObjPrkngCnt      object
objInfrstrObjPrkngCnt         object
objInfrstrPlaygrndCnt         object
objInfrstrRampFlg              int64
o

### Cохранение dataframe в excel

In [138]:
filepath=f'D:\\Дима\\Учеба\\Python\\DA.SberUniversity\\Lesson4\\Homework'
filename=f'objDomRF.xlsx'
objdf.to_excel(filepath+'\\'+filename)

### Cохранение dataframe в pickle

In [140]:
filepath=f'D:\\Дима\\Учеба\\Python\\DA.SberUniversity\\Lesson4\\Homework'
filename=f'objDomRF.pkl'
objdf.to_pickle(filepath+'\\'+filename)

### Cохранение dataframe в БД (SQL light)

In [145]:
conn=sqlite3.connect(filepath+'\\'+'objects.db')
cur=conn.cursor()

In [276]:
# создание таблицы objectsproperties
datatypesdic={'int64':'INT','float64':'FLOAT','datetime64[ns]':'DATE','object':'TEXT'}
tablenamecreate='objectsproperties'
sqlcreate='CREATE TABLE IF NOT EXISTS ' + tablenamecreate + '('
for col in objdf.columns:
    sqlcreate += str(col) + ' ' + str(datatypesdic[str(objdf.dtypes.loc[col])]) + ', '
sqlcreate = sqlcreate[:-2] + ')'


cur.execute(sql)
conn.commit()

In [328]:
# заполнение таблицы objectsproperties
tablenameinsert='objectsproperties'
sqlinsert='INSERT INTO ' + tablenameinsert + ' VALUES (' + '?,'*len(objdf.columns)
sqlinsert=sqlinsert[:-1]+');'

cur.executemany(sqlinsert,objdf.values)
conn.commit()

In [329]:
# проверка корректности команды insert
sqlsel1='SELECT * FROM ' + tablenameinsert + ' limit 10'
cur.execute(sqlsel1)
sqlsel1df=pd.DataFrame(cur.fetchall(),columns=objdf.columns)
sqlsel1df

,objId,objLivingType,objStage,objDateFullReady,objRegion,objAddress,objLkLatitude,objLkLongitude,objNonlivFirstFloor,objCompensFundFlg,...,objLivCeilingHeight,objLkClassDesc,objLkFinishTypeDesc,objLkFreePlanDesc,objNonlivElemCnt,objPriceAvg,objProblemFlg,objSquareLiving,objStatus,wallMaterialShortDesc
0,13398,Жилое,Строится,2024-12-31,Тюменская область,"г Тюмень, ул Профсоюзная, д. 56",57.1519,65.5544,0,0,...,2.96,Элитный,Без отделки,Нет,48,47886.0,0,4812.98,0,Другое
1,13383,Жилое,Строится,2022-06-30,Тюменская область,г Тюмень,57.1530,65.5343,1,1,...,2.8,Элитный,По желанию заказчика,Есть,10,53109.0,0,6289.98,0,Другое
2,13397,Жилое,Строится,2024-12-31,Тюменская область,"г Тюмень, ул Профсоюзная, д. 56",57.1519,65.5544,0,0,...,2.96,Элитный,Без отделки,Нет,70,45523.0,0,6248.51,0,Другое
3,13391,Жилое,Строится,2022-06-30,Тюменская область,"г Тюмень, ул Профсоюзная",57.1541,65.5595,1,1,...,2.8,Элитный,По желанию заказчика,Есть,26,55893.0,0,4293.67,0,Другое
4,23682,Жилое,Строится,2022-06-30,Санкт-Петербург город,д Агалатово,60.2152,30.3089,0,1,...,2.73,Типовой,Без отделки,Нет,1,54859.0,0,2748.15,0,Блоки
5,36321,Жилое,Строится,2022-06-30,Приморский край,"г Владивосток, район Ленинский, ул 10-я Рабоча...",43.1258,131.9341,0,0,...,2.7,Комфорт,Без отделки,Есть,10,131687.0,0,8002.40,0,Монолит
6,36912,Жилое,Строится,2022-12-31,Ставропольский край,"г Ставрополь, ул Шпаковская, Доваторцев, д. 10...",45.0200,41.9346,1,0,...,2.85,Комфорт,Под чистовую,Нет,10,60573.0,0,8295.20,0,Монолит-кирпич
7,23494,Жилое,Строится,2022-12-31,Санкт-Петербург город,"гп Янино-1, улица 7-ая линия, вл. Участок 1",59.9454,30.5612,1,1,...,2.55,Типовой,Без отделки,Нет,3,38582.0,0,10991.85,0,Кирпич
8,27476,Жилое,Строится,2022-06-30,Москва город,"п Коммунарка, д. 17, корпус 4",55.5632,37.5025,1,0,...,2.99,Комфорт,Без отделки,Нет,25,200581.0,0,5589.50,0,Монолит-кирпич
9,10891,Жилое,Строится,2022-09-30,Волгоградская область,"г Вологда, ул Медуницинская, д. 7",59.1985,39.8580,0,1,...,2.7,Комфорт,Без отделки,Есть,3,41999.0,0,12853.77,0,Панель


# Задание 2

* Выберите регион
* Сравните среднюю цену квадратного метра в городах этого региона
* Определите количество строящихся объектов и квартир в регионах
* Определите объем строящегося жилья в квадратных метрах

In [369]:
objdf58reg=objdf[objdf['objRegion']=='Пензенская область']
objdf58reg.head()

,objId,objLivingType,objStage,objDateFullReady,objRegion,objAddress,objLkLatitude,objLkLongitude,objNonlivFirstFloor,objCompensFundFlg,...,objLivCeilingHeight,objLkClassDesc,objLkFinishTypeDesc,objLkFreePlanDesc,objNonlivElemCnt,objPriceAvg,objProblemFlg,objSquareLiving,objStatus,wallMaterialShortDesc
788,41106,Жилое,Строится,2023-06-30,Пензенская область,"с Засечное, ул Новая, строение 12",53.1293,45.0343,1,0,...,2.50,Комфорт,Другое,Нет,11,65913.0,0,9148.5,0,Кирпич
789,38847,Жилое,Строится,2022-12-31,Пензенская область,"с Засечное, ул Новая, д. 17",53.1299,45.0329,1,0,...,2.50,Типовой,Другое,Нет,15,55306.0,0,11540.7,0,Кирпич
1313,19466,Жилое,Строится,2022-06-30,Пензенская область,"г Пенза, район Октябрьский, улица Ладожская, д...",53.2234,44.8775,0,1,...,2.70,Типовой,Другое,Нет,0,45514.0,0,5704.6,0,Другое
1452,36089,Жилое,Строится,2023-03-31,Пензенская область,"г Пенза, район Октябрьский, проспект Победы, д...",53.2304,44.9363,1,0,...,2.72,Комфорт,Под чистовую,Нет,33,63160.0,0,19018.5,0,Монолит-кирпич
1453,39153,Жилое,Строится,2022-12-31,Пензенская область,"г Пенза, ул Окружная, д. 16",53.1668,44.9760,1,0,...,2.70,Комфорт,Другое,Нет,2,64671.0,0,8214.0,0,Монолит-кирпич


In [370]:
objdf58reg['objCity']=objdf58reg['objAddress'].map(lambda s: s.split(',')[0])
objdf58reg['objCity'].unique()

C:\Users\iraid\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array(['с Засечное', 'г Пенза', 'г Заречный', 'Пенза', 'г Кузнецк'],
      dtype=object)

In [371]:
objdf58reg['objCity']=objdf58reg['objCity'].map(lambda g: 'г Пенза' if g=='Пенза' else g)

C:\Users\iraid\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [372]:
objdf58reg['objCity'].unique()

array(['с Засечное', 'г Пенза', 'г Заречный', 'г Кузнецк'], dtype=object)

In [373]:
info58reg=objdf58reg[['objCity','objPriceAvg','objFlatCnt','objFlatSq']].groupby(by='objCity',as_index=False).agg(['count','mean','sum'])

In [374]:
info58reg.reset_index(inplace=True)

In [375]:
info58reg

objCity objPriceAvg                          objFlatCnt              \
                    count          mean        sum      count        mean   
0  г Заречный           2  47019.500000    94039.0          2   69.000000   
1   г Кузнецк           1  51023.000000    51023.0          1   53.000000   
2     г Пенза          21  60923.809524  1279400.0         23  289.956522   
3  с Засечное           4  59356.500000   237426.0          4  267.250000   

        objFlatSq                           
    sum     count          mean        sum  
0   138         2   4391.430000    8782.86  
1    53         1   3518.430000    3518.43  
2  6669        23  15149.431739  348436.93  
3  1069         4  13796.852500   55187.41

In [380]:
meanPrice58Reg=info58reg.melt(value_vars=[('objPriceAvg','mean')])['value']

countObj58Reg=info58reg.melt(value_vars=[('objFlatCnt','count')])['value']

sumFlatCnt58Reg=info58reg.melt(value_vars=[('objFlatCnt','sum')])['value']

sumFlatSq58Reg=info58reg.melt(value_vars=[('objFlatSq','sum')])['value']

In [388]:
info58regshort=pd.DataFrame({'City':info58reg['objCity'],'CountObjects':countObj58Reg,'MeanPriceSqM':meanPrice58Reg,'TotalFlatsCount':sumFlatCnt58Reg,'TotalFlatsSquare':sumFlatSq58Reg})
info58regshort

,City,CountObjects,MeanPriceSqM,TotalFlatsCount,TotalFlatsSquare
0,г Заречный,2,47019.500000,138,8782.86
1,г Кузнецк,1,51023.000000,53,3518.43
2,г Пенза,23,60923.809524,6669,348436.93
3,с Засечное,4,59356.500000,1069,55187.41
